In [1]:
import pandas as pd
from keras.models import load_model, Sequential
from sklearn import preprocessing
import matplotlib.pyplot as plt
from keras.layers import Dense
from keras import backend as K
import sklearn.metrics as skm, math

C:\Program Files\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis = -1))
def r_square(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return (1 - SS_res/(SS_tot + K.epsilon()))

### With Implied Volatility

In [25]:
#Load saved model
model = load_model('model13.h5', custom_objects={'rmse': rmse, 'r_square': r_square})
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 64)                512       
_________________________________________________________________
dense_6 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_7 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 65        
Total params: 17,153
Trainable params: 17,153
Non-trainable params: 0
_________________________________________________________________


In [30]:
df = pd.read_csv('Options_R_test.csv')
df = df[df['underlying_price'].notnull()]
df.head()

,ticker,date,exdate,maturity,strike_price,best_offer,impl_volatility,realized_vol,underlying_price,interest_rate,cp_flag_C,cp_flag_P
0,AXP,2016-01-04,2016-01-08,4,100.0,0.13,1.695791,0.208476,67.59,0.0022,1,0
1,AXP,2016-01-04,2016-01-08,4,105.0,0.13,1.869624,0.208476,67.59,0.0022,1,0
2,AXP,2016-01-04,2016-01-08,4,66.0,1.70,0.238987,0.208476,67.59,0.0022,1,0
3,AXP,2016-01-04,2016-01-08,4,66.5,1.31,0.246213,0.208476,67.59,0.0022,1,0
4,AXP,2016-01-04,2016-01-08,4,67.0,0.95,0.242758,0.208476,67.59,0.0022,1,0


In [80]:
y = df['best_offer'].values
X = df[['maturity', 'strike_price', 'impl_volatility', 'underlying_price', 'cp_flag_C', 'cp_flag_P', 'interest_rate']]
X.head()

,maturity,strike_price,impl_volatility,underlying_price,cp_flag_C,cp_flag_P,interest_rate
0,4,100.0,1.695791,67.59,1,0,0.0022
1,4,105.0,1.869624,67.59,1,0,0.0022
2,4,66.0,0.238987,67.59,1,0,0.0022
3,4,66.5,0.246213,67.59,1,0,0.0022
4,4,67.0,0.242758,67.59,1,0,0.0022


In [81]:
# Normalize data to improve accuracy
X = preprocessing.normalize(X)
y = y.reshape(-1,1)

In [82]:
pred = model.predict(X)
print('Prediction:\n', pred[:10])
print('\n')
print('Actual:\n', y[:10])

Prediction:
 [[-0.63037866]
 [-1.2603066 ]
 [ 1.7310808 ]
 [ 1.4650846 ]
 [ 1.1808021 ]
 [ 0.9605215 ]
 [ 0.7537082 ]
 [ 0.541632  ]
 [ 0.43626946]
 [ 0.3767013 ]]


Actual:
 [[0.13]
 [0.13]
 [1.7 ]
 [1.31]
 [0.95]
 [0.66]
 [0.44]
 [0.28]
 [0.17]
 [0.1 ]]


In [83]:
# Compare prediction accuracy to the actual values
print("Mean absolute error (MAE):      %f" % skm.mean_absolute_error(y,pred))
print("Mean squared error (MSE):       %f" % skm.mean_squared_error(y,pred))
print("Root mean squared error (RMSE): %f" % math.sqrt(skm.mean_squared_error(y,pred)))
print("R square (R^2):                 %f" % skm.r2_score(y,pred))

Mean absolute error (MAE):      1.538255
Mean squared error (MSE):       46.119662
Root mean squared error (RMSE): 6.791146
R square (R^2):                 0.791324


### With Realized Volatility

In [84]:
y2 = df['best_offer'].values
X2 = df[['maturity', 'strike_price', 'realized_vol', 'underlying_price', 'cp_flag_C', 'cp_flag_P', 'interest_rate']]
X2.head()

,maturity,strike_price,realized_vol,underlying_price,cp_flag_C,cp_flag_P,interest_rate
0,4,100.0,0.208476,67.59,1,0,0.0022
1,4,105.0,0.208476,67.59,1,0,0.0022
2,4,66.0,0.208476,67.59,1,0,0.0022
3,4,66.5,0.208476,67.59,1,0,0.0022
4,4,67.0,0.208476,67.59,1,0,0.0022


In [85]:
X2 = preprocessing.normalize(X2)
y2 = y2.reshape(-1,1)

In [89]:
pred2 = model.predict(X2)
print('Prediction:\n', pred2[:10])
print('\n')
print('Actual:\n', y2[:10])

Prediction:
 [[-0.9947671 ]
 [-0.9767634 ]
 [ 1.618005  ]
 [ 1.3256259 ]
 [ 1.0366731 ]
 [ 0.7992919 ]
 [ 0.5684584 ]
 [ 0.3945852 ]
 [ 0.3358479 ]
 [ 0.27181274]]


Actual:
 [[0.13]
 [0.13]
 [1.7 ]
 [1.31]
 [0.95]
 [0.66]
 [0.44]
 [0.28]
 [0.17]
 [0.1 ]]


In [90]:
print("Mean absolute error (MAE):      %f" % skm.mean_absolute_error(y2,pred2))
print("Mean squared error (MSE):       %f" % skm.mean_squared_error(y2,pred2))
print("Root mean squared error (RMSE): %f" % math.sqrt(skm.mean_squared_error(y2,pred2)))
print("R square (R^2):                 %f" % skm.r2_score(y2,pred2))

Mean absolute error (MAE):      1.245420
Mean squared error (MSE):       11.366141
Root mean squared error (RMSE): 3.371371
R square (R^2):                 0.948572


#### Predict special ticker which was used to train the model

In [91]:
MCD = df[df.ticker == 'MCD']
y_MCD = MCD['best_offer'].values
X_MCD = MCD[['maturity', 'strike_price', 'realized_vol', 'underlying_price', 'cp_flag_C', 'cp_flag_P', 'interest_rate']]

In [92]:
X_MCD = preprocessing.normalize(X_MCD)
y_MCD = y_MCD.reshape(-1,1)

In [93]:
pred_MCD = model.predict(X_MCD)
print('Prediction:\n', pred_MCD[:10])
print('\n')
print('Actual:\n', y_MCD[:10])

Prediction:
 [[ 2.3215168 ]
 [ 1.3978105 ]
 [ 0.800794  ]
 [ 0.36472577]
 [ 0.1758582 ]
 [ 0.08003968]
 [ 0.0376575 ]
 [ 0.03258914]
 [-0.00774258]
 [-0.04725128]]


Actual:
 [[3.1 ]
 [2.23]
 [1.49]
 [0.9 ]
 [0.49]
 [0.25]
 [0.13]
 [0.07]
 [0.05]
 [0.13]]


In [94]:
print("Mean absolute error (MAE):      %f" % skm.mean_absolute_error(y_MCD,pred_MCD))
print("Mean squared error (MSE):       %f" % skm.mean_squared_error(y_MCD,pred_MCD))
print("Root mean squared error (RMSE): %f" % math.sqrt(skm.mean_squared_error(y_MCD,pred_MCD)))
print("R square (R^2):                 %f" % skm.r2_score(y_MCD,pred_MCD))

Mean absolute error (MAE):      1.064137
Mean squared error (MSE):       4.993369
Root mean squared error (RMSE): 2.234585
R square (R^2):                 0.965929


In [95]:
GS = df[df.ticker == 'GS']
y_GS = GS['best_offer'].values
X_GS = GS[['maturity', 'strike_price', 'realized_vol', 'underlying_price', 'cp_flag_C', 'cp_flag_P', 'interest_rate']]

In [96]:
X_GS = preprocessing.normalize(X_GS)
pred_GS = model.predict(X_GS)
print('Prediction:\n', pred_GS[:10])
print('\n')
print('Actual:\n', y_GS[:10])

Prediction:
 [[5.9363313 ]
 [4.097701  ]
 [2.8001225 ]
 [1.9598825 ]
 [1.1927996 ]
 [0.6488946 ]
 [0.39673084]
 [0.21955419]
 [0.08179379]
 [0.03232047]]


Actual:
 [5.45 3.5  2.02 1.04 0.47 0.21 0.12 0.07 0.13 0.03]


In [97]:
print("Mean absolute error (MAE):      %f" % skm.mean_absolute_error(y_GS,pred_GS))
print("Mean squared error (MSE):       %f" % skm.mean_squared_error(y_GS,pred_GS))
print("Root mean squared error (RMSE): %f" % math.sqrt(skm.mean_squared_error(y_GS,pred_GS)))
print("R square (R^2):                 %f" % skm.r2_score(y_GS,pred_GS))

Mean absolute error (MAE):      2.462353
Mean squared error (MSE):       37.482205
Root mean squared error (RMSE): 6.122271
R square (R^2):                 0.936342


#### Predict special ticker which was not used to train the model

In [59]:
AAPL = pd.read_csv('Options2016AAPL_withReaVo.csv')
AAPL.head()

,Unnamed: 0,Strike Price,Lowest Closing Ask Across All Exchanges,Underlying Price,maturity,"C=Call, P=Put_C","C=Call, P=Put_P",interest_rate,days,volatility
0,0,100.0,5.45,105.35,4,1,0,0.0022,30,0.218791
1,1,101.0,4.50,105.35,4,1,0,0.0022,30,0.218791
2,2,102.0,3.65,105.35,4,1,0,0.0022,30,0.218791
3,3,103.0,2.79,105.35,4,1,0,0.0022,30,0.218791
4,4,104.0,2.06,105.35,4,1,0,0.0022,30,0.218791


In [70]:
AAPL = AAPL[AAPL['interest_rate'].notnull()]

In [71]:
y_AAPL = AAPL['Lowest  Closing Ask Across All Exchanges'].values
X_AAPL = AAPL[['maturity', 'Strike Price', 'volatility', 'Underlying Price', 'C=Call, P=Put_C', 'C=Call, P=Put_P', 'interest_rate']]

In [72]:
X_AAPL = preprocessing.normalize(X_AAPL)
y_AAPL = y_AAPL.reshape(-1,1)

In [73]:
pred4 = model.predict(X_AAPL)
print('Prediction:\n', pred4[:10])
print('\n')
print('Actual:\n', y_AAPL[:10])

Prediction:
 [[5.4338818 ]
 [4.5494175 ]
 [3.6288536 ]
 [2.7997174 ]
 [2.1064494 ]
 [1.4581447 ]
 [0.99268407]
 [0.5368069 ]
 [0.24624091]
 [0.1311115 ]]


Actual:
 [[5.45]
 [4.5 ]
 [3.65]
 [2.79]
 [2.06]
 [1.43]
 [0.93]
 [0.56]
 [0.32]
 [0.18]]


In [74]:
print("Mean absolute error (MAE):      %f" % skm.mean_absolute_error(y_AAPL,pred4))
print("Mean squared error (MSE):       %f" % skm.mean_squared_error(y_AAPL,pred4))
print("Root mean squared error (RMSE): %f" % math.sqrt(skm.mean_squared_error(y_AAPL,pred4)))
print("R square (R^2):                 %f" % skm.r2_score(y_AAPL,pred4))

Mean absolute error (MAE):      3.518400
Mean squared error (MSE):       81.153715
Root mean squared error (RMSE): 9.008536
R square (R^2):                 0.868447
